In [18]:
import pandas as pd
import numpy as np

#fonction pour transformer str en liste
import ast

In [19]:
chemin_bd = r"./bd_ignore/"

In [20]:
df_tmdb = pd.read_csv(chemin_bd + 'tmdb_full.csv')  # Dataset des films
df_names = pd.read_csv(chemin_bd + 'name.basics.tsv', sep='\t')  # Dataset des acteurs
df_title_principals = pd.read_csv(chemin_bd + 'title.principals.tsv', sep='\t')  # Dataset des acteurs

/var/folders/4f/28z_1f6d67b_wy1qrp_rvp6w0000gn/T/ipykernel_35303/1803814735.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmdb = pd.read_csv(chemin_bd + 'tmdb_full.csv')  # Dataset des films


In [21]:
df_tmdb.head(2)

,adult,backdrop_path,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,production_companies_name,production_companies_country
0,False,/dvQj1GBZAZirz1skEEZyWH2ZqQP.jpg,0,['Comedy'],NaN,3924,tt0029927,en,Blondie,Blondie and Dagwood are about to celebrate the...,...,70,['en'],Released,The favorite comic strip of millions at last o...,Blondie,False,7.214,7,['Columbia Pictures'],['US']
1,False,NaN,0,['Adventure'],NaN,6124,tt0011436,de,Der Mann ohne Namen,NaN,...,420,[],Released,NaN,"Peter Voss, Thief of Millions",False,0.000,0,[],[]


In [22]:
# FILTRAGE TMDB

# Nettoyer et convertir la colonne 'release_date' en datetime dans df_tmdb
df_tmdb['release_date'] = pd.to_datetime(df_tmdb['release_date'], errors='coerce')
df_tmdb['year'] = df_tmdb['release_date'].dt.year

# Supprimer les lignes sans 'year' ou 'genres'
df_tmdb = df_tmdb.dropna(subset=['year', 'genres'])

# Filtrer les films des années 2000
df_tmdb = df_tmdb[(df_tmdb['year'] >= 2000) & (df_tmdb['year'] <= 2024)]

# filtrer les genres
df_tmdb = df_tmdb[df_tmdb['genres'].str.contains("|".join(['Action', 'Drama', 'Comedy', 'Horror', 'Family', 'Adventure', 'Animation', 'Thriller']))]

# Exploser les genres associés (genres)
df_tmdb = df_tmdb.assign(genres=df_tmdb['genres'].str.split(','))
df_tmdb = df_tmdb.explode('genres')  # Une ligne par genre associé

# Merge avec le dataset des acteurs sur 'imdb_id'
df_tmdb['imdb_id'] = df_tmdb['imdb_id'].str.strip()  # Nettoyer d'éventuels espaces

# Enlever les []
df_tmdb['genres']=df_tmdb['genres'].apply(lambda x : str(x).replace('[','').replace(']',''))
df_tmdb['genres']=df_tmdb['genres'].apply(lambda x : str(x).replace("'",'').replace("'",''))

df_tmdb = df_tmdb.drop(columns=['adult', 'backdrop_path','id','original_language', 'original_title', 'overview','production_countries','spoken_languages', 'status', 'tagline','video','production_companies_name','production_companies_country'])

df_tmdb.to_csv('df_tmdb2.csv', index = False)


In [23]:
df_tmdb2 = pd.read_csv('df_tmdb2.csv')

In [24]:
df_tmdb2.head()

,budget,genres,homepage,imdb_id,popularity,poster_path,release_date,revenue,runtime,title,vote_average,vote_count,year
0,0,Drama,NaN,tt0425473,1.930,NaN,2004-09-02,0,15,Sunday in August,7.738,21,2004.0
1,94000000,Animation,http://movies.disney.com/finding-nemo,tt0266543,68.898,/5lc6nQc0VhWFYFbNv016xze8Jvy.jpg,2003-05-30,940335536,100,Finding Nemo,7.823,17625,2003.0
2,94000000,Family,http://movies.disney.com/finding-nemo,tt0266543,68.898,/5lc6nQc0VhWFYFbNv016xze8Jvy.jpg,2003-05-30,940335536,100,Finding Nemo,7.823,17625,2003.0
3,12800000,Drama,NaN,tt0168629,13.781,/8Wdd3fQfbbQeoSfWpHrDfaFNhBU.jpg,2000-06-30,40031879,141,Dancer in the Dark,7.881,1555,2000.0
4,12800000,Crime,NaN,tt0168629,13.781,/8Wdd3fQfbbQeoSfWpHrDfaFNhBU.jpg,2000-06-30,40031879,141,Dancer in the Dark,7.881,1555,2000.0


In [25]:
df_title_principals.head(2)

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N


In [26]:
df_names.head(2)

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"actor,miscellaneous,producer","tt0050419,tt0072308,tt0053137,tt0027125"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"


In [27]:
# prends que les acteurs du df de title principals

df_title_principals = df_title_principals[df_title_principals['category'].isin(['actor', 'actress'])]
df_title_principals['characters'] = df_title_principals['characters'].dropna()
df_title_principals= df_title_principals.dropna(subset=['characters'])
df_title_principals.reset_index(inplace=True, drop=True)
df_title_principals = df_title_principals[['tconst', 'nconst']]

In [28]:
df_tp_tmdb = pd.merge(df_tmdb, df_title_principals, left_on='imdb_id', right_on='tconst', how='left')

In [29]:
# prend que le nom des acteurs et la colonne nconst qui servira à faire le merge

df_names2 = df_names.copy()
df_names2 = df_names2[['nconst', 'primaryName']]
df_names2.head(1)

,nconst,primaryName
0,nm0000001,Fred Astaire


In [30]:
tp_tmdb_n = pd.merge(df_tp_tmdb, df_names2, left_on='nconst', right_on='nconst', how='left')

In [32]:
tp_tmdb_n.to_csv(chemin_bd + 'resultat/tp_tmdb_n.csv', index = False)

In [34]:
tp_tmdb_n2 = pd.read_csv(chemin_bd + 'resultat/tp_tmdb_n.csv')

In [36]:
tp_tmdb_n3 = tp_tmdb_n2.copy()

In [37]:
# Nettoyer et convertir la colonne 'release_date' en datetime dans df_tmdb
tp_tmdb_n3['release_date'] = pd.to_datetime(tp_tmdb_n3['release_date'], errors='coerce')
tp_tmdb_n3['year'] = tp_tmdb_n3['release_date'].dt.year

# Supprimer les lignes sans 'year' ou 'genres'
tp_tmdb_n3 = tp_tmdb_n3.dropna(subset=['year', 'genres'])

# Filtrer les films des années 2000
tp_tmdb_n3 = tp_tmdb_n3[(tp_tmdb_n3['year'] >= 2000) & (tp_tmdb_n3['year'] <= 2024)]

# filtrer les genres
tp_tmdb_n3 = tp_tmdb_n3[tp_tmdb_n3['genres'].str.contains("|".join(['Action', 'Drama', 'Comedy', 'Horror', 'Family', 'Adventure', 'Animation', 'Thriller']))]

# Exploser les genres associés (genres)
tp_tmdb_n3 = tp_tmdb_n3.assign(genres=tp_tmdb_n3['genres'].str.split(','))
tp_tmdb_n3 = tp_tmdb_n3.explode('genres')  # Une ligne par genre associé

# # Merge avec le dataset des acteurs sur 'imdb_id'
# df_tmdb['imdb_id'] = df_tmdb['imdb_id'].str.strip()  # Nettoyer d'éventuels espaces

# Enlever les []
tp_tmdb_n3['genres']=tp_tmdb_n3['genres'].apply(lambda x : str(x).replace('[','').replace(']',''))
tp_tmdb_n3['genres']=tp_tmdb_n3['genres'].apply(lambda x : str(x).replace("'",'').replace("'",''))

# df_tmdb2 = df_tmdb.copy()
# df_merged = pd.merge(df_tmdb, df_names, left_on='imdb_id', right_on='knownForTitles', how='left')

# Ajouter les noms des acteurs groupés par film
tp_tmdb_n3['actors'] = tp_tmdb_n3.groupby('imdb_id')['primaryName'].transform(lambda x: ', '.join(x.dropna()))
tp_tmdb_n3 = tp_tmdb_n3.drop_duplicates(subset=['imdb_id'])  # Supprimer les doublons

tp_tmdb_n3['actors'] = tp_tmdb_n3['actors'].apply(lambda x: list(x.split(',')))

# Sélection des colonnes pertinentes pour le modèle
features = ['popularity', 'vote_average', 'vote_count', 'budget', 'revenue', 'runtime']
df_filtered_new = tp_tmdb_n3[features + ['genres', 'actors', 'title', 'imdb_id', 'year']].fillna(0)


In [38]:
df_filtered_new['actors'] = df_filtered_new['actors'].apply(lambda x: set(x))
df_filtered_new['actors'] = df_filtered_new['actors'].apply(lambda x: str(x).replace('{','').replace('}',''))
df_filtered_new['actors'] = df_filtered_new['actors'].apply(lambda x: str(x).replace("'",'').replace("'",''))
#df_filtered_new['actors'] = df_filtered_new['actors'].apply(lambda x: str(x).split(','))

In [39]:
df_filtered_new.head()

,popularity,vote_average,vote_count,budget,revenue,runtime,genres,actors,title,imdb_id,year
0,1.930,7.738,21,0,0,15,Drama,"Milton Welsh, Rita Lengyel",Sunday in August,tt0425473,2004
2,68.898,7.823,17625,94000000,940335536,100,Animation,"Albert Brooks, Alexander Gould, Austin Pendl...",Finding Nemo,tt0266543,2003
24,13.781,7.881,1555,12800000,40031879,141,Drama,"Jean-Marc Barr, Joel Grey, Siobhan Fallon H...",Dancer in the Dark,tt0168629,2000
44,7.178,5.815,232,0,6593579,87,Horror,"Maria Bello, Gwenyth Petty, Abigail Stone, ...",The Dark,tt0411267,2005
74,11.068,5.880,403,2500000,12300000,106,Drama,"Scott Speedman, Mark Ruffalo, Leonor Watlin...",My Life Without Me,tt0314412,2003


In [40]:
# on les met dans un fichier plus court pour la recherche
df_filtered_new.to_csv('df_filtered_new.csv', index = False)

In [41]:
df_filtered_newactor = pd.read_csv('df_filtered_new.csv')

In [42]:
df_filtered_newactor.head()

,popularity,vote_average,vote_count,budget,revenue,runtime,genres,actors,title,imdb_id,year
0,1.930,7.738,21,0,0,15,Drama,"Milton Welsh, Rita Lengyel",Sunday in August,tt0425473,2004
1,68.898,7.823,17625,94000000,940335536,100,Animation,"Albert Brooks, Alexander Gould, Austin Pendl...",Finding Nemo,tt0266543,2003
2,13.781,7.881,1555,12800000,40031879,141,Drama,"Jean-Marc Barr, Joel Grey, Siobhan Fallon H...",Dancer in the Dark,tt0168629,2000
3,7.178,5.815,232,0,6593579,87,Horror,"Maria Bello, Gwenyth Petty, Abigail Stone, ...",The Dark,tt0411267,2005
4,11.068,5.880,403,2500000,12300000,106,Drama,"Scott Speedman, Mark Ruffalo, Leonor Watlin...",My Life Without Me,tt0314412,2003


In [43]:
#df_filtered_new['actors'] = df_filtered_new['actors'].apply(lambda x: str(x).split(','))

df_filtered_newactor['first_two_actors'] = df_filtered_newactor['actors'].apply(
    lambda x: ','.join(x.split(',')[:3]) if isinstance(x, str) else None
)

# on met en liste dans une nouvelle colonne 
df_filtered_newactor['two_actors'] = df_filtered_newactor['first_two_actors'].apply(lambda x: str(x).split(","))
df_filtered_newactor['two_actors'] = df_filtered_newactor['two_actors'].apply(lambda x: set(x))

# Enlever les {}
df_filtered_newactor['two_actors'] = df_filtered_newactor['two_actors'].apply(lambda x: str(x).replace('{','').replace('}',''))
df_filtered_newactor['two_actors'] = df_filtered_newactor['two_actors'].apply(lambda x: str(x).replace("'  ",'').replace("'",''))

#mettre colonne en liste
df_filtered_newactor['two_actors'] = df_filtered_newactor['two_actors'].apply(lambda x: x.split(','))

# on supprime les colonnes inutiles
df_filtered_newactor = df_filtered_newactor.drop(columns =['first_two_actors'])

In [44]:
df_filtered_newactor.to_csv('df_filtered_newactor.csv', index = False)

In [45]:
df_filtered_newactor2 = pd.read_csv('df_filtered_newactor.csv')

In [46]:
df_filtered_newactor2.head()

,popularity,vote_average,vote_count,budget,revenue,runtime,genres,actors,title,imdb_id,year,two_actors
0,1.930,7.738,21,0,0,15,Drama,"Milton Welsh, Rita Lengyel",Sunday in August,tt0425473,2004,"[' Milton Welsh', ' Rita Lengyel']"
1,68.898,7.823,17625,94000000,940335536,100,Animation,"Albert Brooks, Alexander Gould, Austin Pendl...",Finding Nemo,tt0266543,2003,"['Albert Brooks', ' Austin Pendleton', ' Alexa..."
2,13.781,7.881,1555,12800000,40031879,141,Drama,"Jean-Marc Barr, Joel Grey, Siobhan Fallon H...",Dancer in the Dark,tt0168629,2000,"['Joel Grey', ' Siobhan Fallon Hogan', ' Jean..."
3,7.178,5.815,232,0,6593579,87,Horror,"Maria Bello, Gwenyth Petty, Abigail Stone, ...",The Dark,tt0411267,2005,"['Gwenyth Petty', ' Maria Bello', ' Abigail S..."
4,11.068,5.880,403,2500000,12300000,106,Drama,"Scott Speedman, Mark Ruffalo, Leonor Watlin...",My Life Without Me,tt0314412,2003,"['Leonor Watling', ' Mark Ruffalo', ' Scott S..."


In [47]:
#### Jonathan

In [48]:
# # liste des acteurs uniques

# list_actor = df_filtered['two_actors'].apply(lambda x: [actor.strip() for actor in ast.literal_eval(x)] if pd.notna(x) else [])
# # on convertit str en list
 
# actor_unique = set()
# for actor in list_actor :
#     actor_unique.update(actor)
# # on parcourt ttes les list d'acteurs et on les mets dans actor_unique

# list_actor_unique = list(actor_unique)

# list_actor_unique

In [49]:
# # filtre avec le nom d'un acteur

# actor_search  = 'Thomas Vicari'
# df_filtered[df_filtered['two_actors'].apply(lambda x: actor_search in [a.strip() for a in ast.literal_eval(x)] if pd.notna(x) else False )]

In [50]:
        #### jonathan affichage des images

        # for idx, res in resultats.iterrows():
        #     title = res.get('title', 'Titre inconnu')
        #     #poster_path = res['poster_path']
        #     #imdb_id = res['imdb_id']
        #     poster_path = res.get('poster_path')
        #     imdb_id = res.get('imdb_id')

        #     col_courante = cols[idx % 3]
        #     # Ajouter les informations dans les colonnes
        #     with col_courante :
        #         if pd.notna(poster_path) :
        #             # Afficher l'affiche du film
        #             #st.image(f"https://image.tmdb.org/t/p/w500{poster_path}", width=150, caption=title)
        #             st.write(f"![poster](https://image.tmdb.org/t/p/w500{poster_path}")
        #         else:
        #             st.write(f"**{title}** (Aucune affiche disponible)")

        #         # Lien vers IMDb
        #         if imdb_id and str(imdb_id).strip():
        #             st.write(f"[Voir sur IMDb](https://www.imdb.com/title/{imdb_id}/)")

    # st.write(f"### Films sélectionnés :")
    # st.write(resultats)

In [51]:
# pour deployer sur streamlit share

# import gdown
# import pandas as pd

# # L'URL de partage Google Drive
# # url à utiliser : https://drive.google.com/uc?id= ... ensuite tu mets l'ID du fichier
# url = "https://drive.google.com/uc?id=1QF-nUGIoyo8eEecSOV2iDmMzfmU2X_eM"

# # Il faut récupérer l'ID du fichier
# # https://drive.google.com/file/d/1QF-nUGIoyo8eEecSOV2iDmMzfmU2X_eM/view?usp=share_link
# # 1QF-nUGIoyo8eEecSOV2iDmMzfmU2X_eM

# # Télécharger le fichier
# gdown.download(url, 'mon_fichier.csv', quiet=False)

# # Lire le CSV avec pandas
# df = pd.read_csv('mon_fichier.csv')

# df.head()